In [46]:
%matplotlib inline 

import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import seaborn as sns 
import scipy.stats as stats
import shapefile as shp
import geopandas as gpd
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

## USDA Dataset

In [47]:
# read csv file accidents.csv 
df = pd.read_csv('fooddesert.csv')
df.head()

,CensusTract,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,1001020100,Alabama,Autauga,1,1912,693,0,0,0.000000,0,...,221,1622,217,14,0,14,45,44,26,112
1,1001020200,Alabama,Autauga,1,2170,743,0,181,0.083410,0,...,214,888,1217,5,0,5,55,75,87,202
2,1001020300,Alabama,Autauga,1,3373,1256,0,0,0.000000,0,...,439,2576,647,17,5,11,117,87,108,120
3,1001020400,Alabama,Autauga,1,4386,1722,0,0,0.000000,0,...,904,4086,193,18,4,11,74,85,19,82
4,1001020500,Alabama,Autauga,1,10766,4082,0,181,0.016812,0,...,1126,8666,1437,296,9,48,310,355,198,488


In [48]:
df.shape

(72864, 147)

In [49]:
list(df.columns)

['CensusTract',
 'State',
 'County',
 'Urban',
 'POP2010',
 'OHU2010',
 'GroupQuartersFlag',
 'NUMGQTRS',
 'PCTGQTRS',
 'LILATracts_1And10',
 'LILATracts_halfAnd10',
 'LILATracts_1And20',
 'LILATracts_Vehicle',
 'HUNVFlag',
 'LowIncomeTracts',
 'PovertyRate',
 'MedianFamilyIncome',
 'LA1and10',
 'LAhalfand10',
 'LA1and20',
 'LATracts_half',
 'LATracts1',
 'LATracts10',
 'LATracts20',
 'LATractsVehicle_20',
 'LAPOP1_10',
 'LAPOP05_10',
 'LAPOP1_20',
 'LALOWI1_10',
 'LALOWI05_10',
 'LALOWI1_20',
 'lapophalf',
 'lapophalfshare',
 'lalowihalf',
 'lalowihalfshare',
 'lakidshalf',
 'lakidshalfshare',
 'laseniorshalf',
 'laseniorshalfshare',
 'lawhitehalf',
 'lawhitehalfshare',
 'lablackhalf',
 'lablackhalfshare',
 'laasianhalf',
 'laasianhalfshare',
 'lanhopihalf',
 'lanhopihalfshare',
 'laaianhalf',
 'laaianhalfshare',
 'laomultirhalf',
 'laomultirhalfshare',
 'lahisphalf',
 'lahisphalfshare',
 'lahunvhalf',
 'lahunvhalfshare',
 'lasnaphalf',
 'lasnaphalfshare',
 'lapop1',
 'lapop1share',
 

In [50]:
# filter DC data only 
df_dc = df[df['State'].isin(['District of Columbia'])].set_index('CensusTract')

# check for null vals
df_dc.isnull().sum()

State                   0
County                  0
Urban                   0
POP2010                 0
OHU2010                 0
GroupQuartersFlag       0
NUMGQTRS                0
PCTGQTRS                0
LILATracts_1And10       0
LILATracts_halfAnd10    0
LILATracts_1And20       0
LILATracts_Vehicle      0
HUNVFlag                0
LowIncomeTracts         0
PovertyRate             0
MedianFamilyIncome      0
LA1and10                0
LAhalfand10             0
LA1and20                0
LATracts_half           0
LATracts1               0
LATracts10              0
LATracts20              0
LATractsVehicle_20      0
LAPOP1_10               0
LAPOP05_10              0
LAPOP1_20               0
LALOWI1_10              0
LALOWI05_10             0
LALOWI1_20              0
                       ..
lawhite20               0
lawhite20share          0
lablack20               0
lablack20share          0
laasian20               0
laasian20share          0
lanhopi20               0
lanhopi20sha

In [51]:
# shape of df post filtering 
df_dc.shape

(179, 146)

In [54]:
# merging updated race df with original dataframe
df_all = pd.merge(df_dc, df_race, how='inner', left_on=None, right_on=None,
         left_index=True, right_index=True)

df_all.head(2)

,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,LILATracts_halfAnd10,...,TractHispanic_x,TractHUNV,TractSNAP,TractWhite_y,TractBlack_y,TractAsian_y,TractNHOPI_y,TractAIAN_y,TractOMultir_y,TractHispanic_y
CensusTract,,,,,,,,,,,,,,,,,,,,,
11001000100,District of Columbia,District of Columbia,1,4890,2686,0,24,0.004908,0,0,...,296,634,0,4444,105,201,2,9,129,296
11001000201,District of Columbia,District of Columbia,1,3916,2,1,3908,0.997957,0,1,...,275,0,0,2940,259,482,5,7,223,275


In [55]:
# create population variable, because the pop2010 var has discrepancies with totals 
df_all['totalpop'] = df_all['TractWhite_y'] + df_all['TractBlack_y'] + df_all['TractAsian_y'] + df_all['TractNHOPI_y'] + df_all['TractAIAN_y'] + df_all['TractOMultir_y'] + df_all['TractHispanic_y']
df_all.head()

,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,LILATracts_halfAnd10,...,TractHUNV,TractSNAP,TractWhite_y,TractBlack_y,TractAsian_y,TractNHOPI_y,TractAIAN_y,TractOMultir_y,TractHispanic_y,totalpop
CensusTract,,,,,,,,,,,,,,,,,,,,,
11001000100,District of Columbia,District of Columbia,1,4890,2686,0,24,0.004908,0,0,...,634,0,4444,105,201,2,9,129,296,5186
11001000201,District of Columbia,District of Columbia,1,3916,2,1,3908,0.997957,0,1,...,0,0,2940,259,482,5,7,223,275,4191
11001000202,District of Columbia,District of Columbia,1,5425,1933,0,1135,0.209217,0,0,...,332,0,4705,159,375,3,5,178,343,5768
11001000300,District of Columbia,District of Columbia,1,6233,2754,0,0,0.000000,0,0,...,408,23,5497,139,352,1,11,233,404,6637
11001000400,District of Columbia,District of Columbia,1,1455,636,0,6,0.004124,0,0,...,51,0,1247,50,103,1,2,53,123,1579
